Initial rough analysis of data

In [7]:
import pandas as pd
pd.set_option('display.max_rows', 500)

df = pd.read_csv('./data/applemobilitytrends-2021-07-27.csv')

df

/Users/claraho/Desktop/apple-app-demo/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,geo_type,region,transportation_type,alternative_name,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,...,2021-07-18,2021-07-19,2021-07-20,2021-07-21,2021-07-22,2021-07-23,2021-07-24,2021-07-25,2021-07-26,2021-07-27
0,country/region,Albania,driving,NaN,NaN,NaN,100.0,95.30,101.43,97.20,...,263.83,243.24,251.90,278.22,297.08,315.51,330.00,349.02,330.53,315.20
1,country/region,Albania,walking,NaN,NaN,NaN,100.0,100.68,98.93,98.46,...,202.00,223.75,226.65,260.72,265.93,284.18,273.39,253.82,291.86,270.02
2,country/region,Argentina,driving,NaN,NaN,NaN,100.0,97.07,102.45,111.21,...,76.00,84.39,104.62,87.91,95.23,116.22,122.68,75.03,79.49,81.46
3,country/region,Argentina,walking,NaN,NaN,NaN,100.0,95.11,101.37,112.67,...,50.82,73.40,92.07,75.92,82.94,100.50,90.21,52.75,70.94,71.21
4,country/region,Australia,driving,AU,NaN,NaN,100.0,102.98,104.21,108.63,...,79.44,78.88,78.63,80.76,84.23,76.98,61.62,72.95,76.87,89.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4686,county,York County,walking,NaN,Pennsylvania,United States,100.0,110.22,121.82,109.72,...,185.47,212.71,210.28,212.38,231.33,268.07,265.41,175.75,219.12,211.16
4687,county,Young County,driving,NaN,Texas,United States,100.0,98.85,97.76,92.21,...,160.91,164.84,168.77,169.24,179.13,213.08,177.17,150.34,181.17,162.47
4688,county,Yuba County,driving,NaN,California,United States,100.0,102.98,103.10,114.08,...,158.89,155.40,171.02,158.63,164.93,188.06,174.85,151.90,159.70,154.49
4689,county,Yuma County,driving,NaN,Arizona,United States,100.0,101.82,106.67,108.93,...,124.58,133.49,121.49,128.24,135.70,157.59,139.03,124.56,128.68,122.29


In [17]:
df.columns

Index(['geo_type', 'region', 'transportation_type', 'alternative_name',
       'sub-region', 'country', '2020-01-13', '2020-01-14', '2020-01-15',
       '2020-01-16',
       ...
       '2021-07-18', '2021-07-19', '2021-07-20', '2021-07-21', '2021-07-22',
       '2021-07-23', '2021-07-24', '2021-07-25', '2021-07-26', '2021-07-27'],
      dtype='object', length=568)

Insert Mobility data

In [16]:
import numpy as np

def preprocess_mobility(df):
    id_vars = {'geo_type', 'region', 'transportation_type', 'alternative_name','sub-region', 'country'}
    value_vars = set(df.columns) - id_vars
    df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='date', value_name = 'mobility_rate')
    df['mobility_rate'] = df['mobility_rate'].replace(np.nan, 0, regex=True)
    df = df.replace(np.nan, '', regex=True)

    return df

In [17]:
def checkTableExists(dbcur, tablename):
    # dbcur = dbcon.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        # dbcur.close()
        return True

    # dbcur.close()
    return False

In [ ]:
import pandas as pd
import pymysql
from dotenv import load_dotenv
from tqdm.notebook import tqdm

def create_mobility_table(df):
    
    load_dotenv()

    # PostgreSQL Database credentials loaded from the .env file
    DATABASE = os.getenv('DATABASE')
    DATABASE_USERNAME = os.getenv('DATABASE_USERNAME')
    DATABASE_PASSWORD = os.getenv('DATABASE_PASSWORD')
    DATABASE_HOST = os.getenv('DATABASE_HOST')

    try:
        con = pymysql.connect(
            database=DATABASE,
            user=DATABASE_USERNAME,
            password=DATABASE_PASSWORD,
            host = DATABASE_HOST)

        cur = con.cursor()
        if checkTableExists(cur, 'mobility') == False:
            print('Creating table....')

            cur.execute("CREATE TABLE mobility(region varchar(255),alternative_name varchar(255),country varchar(255),geo_type varchar(255),sub-region varchar(255),transportation_type varchar(255),date DATE,mobility_rate float)")
            print("Table is created....")

        #loop through the data frame
        for i,row in tqdm(df.iterrows()):
            #here %S means string values 
            sql = "INSERT INTO mobility VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
            # print(sql, tuple(row))
            cur.execute(sql, tuple(row))
            # print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            con.commit()

        # cur.execute("LOAD DATA INFILE 'clean.csv' INTO TABLE mobilityData FIELDS TERMINATED BY ',' IGNORE 1 ROWS")
    except pymysql.Error as e:
        print("Error while connecting to MySQL", e)

In [6]:
df = pd.read_csv('./data/applemobilitytrends-2021-07-27.csv')
df = preprocess_mobility(df)
create_mobility_table(df)

/Users/claraho/Desktop/apple-app-demo/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0it [00:00, ?it/s]

Insert covid data

In [ ]:
import numpy as np

def preprocess_covid(df):
    df = df[['iso_code', 'continent', 'location', 'date','total_cases', 'new_cases']]
    df['total_cases'] = df['total_cases'].replace(np.nan, 0, regex=True)
    df['new_cases'] = df['new_cases'].replace(np.nan, 0, regex=True)
    df = df.replace(np.nan, '', regex=True)

    return df

In [ ]:
import pandas as pd
import pymysql
from dotenv import load_dotenv
from tqdm.notebook import tqdm

def create_covid_table(df):
    
    load_dotenv()

    # PostgreSQL Database credentials loaded from the .env file
    DATABASE = os.getenv('DATABASE')
    DATABASE_USERNAME = os.getenv('DATABASE_USERNAME')
    DATABASE_PASSWORD = os.getenv('DATABASE_PASSWORD')
    DATABASE_HOST = os.getenv('DATABASE_HOST')

    try:
        con = pymysql.connect(
            database=DATABASE,
            user=DATABASE_USERNAME,
            password=DATABASE_PASSWORD,
            host = DATABASE_HOST)

        cur = con.cursor()
        if checkTableExists(cur, 'covid') == False:
            print('Creating table....')

            cur.execute("CREATE TABLE covid(iso_code varchar(255), continent varchar(255), location varchar(255), date date, total_cases float, new_cases float)")

        #loop through the data frame
        for i,row in tqdm(df.iterrows()):
            #here %S means string values 
            sql = "INSERT INTO mobility VALUES (%s,%s,%s,%s,%s,%s)"
            # print(sql, tuple(row))
            cur.execute(sql, tuple(row))
            # print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            con.commit()

        # cur.execute("LOAD DATA INFILE 'clean.csv' INTO TABLE mobilityData FIELDS TERMINATED BY ',' IGNORE 1 ROWS")
    except pymysql.Error as e:
        print("Error while connecting to MySQL", e)

In [ ]:
df = pd.read_csv('./data/owid-covid-data.csv')
df = preprocess_covid(df)
create_covid_table(df)